# ML Foreclosure Binary Classification - 2.5% sample dataset
Only "resolved" outcomes - Either paid off or default 

    target = zeroBalCode (set to object dtype) 
    Binary classification sets zeroBalCode to either "0" or "1" for the 7 possible outcomes
    
    "0" = "Loan Success"
        * 01 = Prepaid or Matured
        * 06 = Repurchased
        * 16 = Reperforming Loan Sale

    "1" =  "Loan Failure" - Most likely loss of money (but not necessarily) one of below conditions:
        * 02 = Third Party Sale
        * 03 = Short Sale
        * 09 = Deed-in-Lieu,REO
        * 15 = Note Sale


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#!pip install pycaret
from imblearn.over_sampling import RandomOverSampler
from pycaret.classification import *
#from pycaret.regression import *
pd.set_option('display.max_columns', None)


# Importing the data - don't use *.ML.csv files!!!
If you have *.MLReady.csv then [click here](#MLReady)

In [3]:
df=pd.read_csv("data/FMAcq2000-19FMac30-sample2pt5_loanAge.csv")
df.head()

,Unnamed: 0,id,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origDate,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,monMatur,mSA,zeroBalCode,deliqGood,deliqBad,deliqMax,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,fredRate,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct
0,2201863,869528717503,B,"BANK OF AMERICA, N.A.",6.500,248000,360,2008-01-01,90.0,90.0,1.0,52.0,C,CA,953,25.0,2.0,681.0,681.0,681.0,127,402.0,33700,16.0,117,10,10.0,6.07,5.48,5.7580,0.107664,4.04,0.430,-1.020,0.7420,0.070840,-0.186131,0.128864
1,2327030,783272249623,R,"BANK OF AMERICA, N.A.",5.750,42000,360,2009-01-01,85.0,85.0,1.0,48.0,P,NY,137,12.0,1.0,755.0,755.0,755.0,54,306.0,13780,6.0,46,7,5.0,5.12,4.96,5.0475,0.032258,2.25,0.630,-0.790,0.7025,0.123047,-0.159274,0.139178
2,918867,494357556458,C,"WITMER FUNDING, LLC",5.875,216000,360,2003-02-01,79.0,79.0,2.0,26.0,R,WI,531,0.0,0.0,753.0,748.0,750.5,138,222.0,33340,1.0,88,49,3.0,5.88,5.79,5.8425,0.015544,4.00,-0.005,-0.085,0.0325,-0.000850,-0.014680,0.005563
3,1570452,586884174908,C,"BANK OF AMERICA, N.A.",5.750,252000,360,2004-07-01,79.0,79.0,1.0,50.0,R,CA,958,0.0,0.0,681.0,681.0,681.0,72,288.0,40900,2.0,37,35,31.0,6.21,5.98,6.0560,0.038462,4.57,-0.460,0.230,-0.3060,-0.074074,0.038462,-0.050528
4,2571240,574324938820,C,OTHER,3.625,102000,180,2013-10-01,80.0,80.0,1.0,29.0,C,AL,351,0.0,0.0,665.0,665.0,665.0,49,131.0,13820,1.0,48,1,1.0,4.28,4.10,4.1920,0.043902,2.66,-0.655,0.475,-0.5670,-0.153037,0.115854,-0.135258


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66582 entries, 0 to 66581
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          66582 non-null  int64  
 1   id                  66582 non-null  int64  
 2   origChannel         66582 non-null  object 
 3   sellerName          66582 non-null  object 
 4   origIntRate         66582 non-null  float64
 5   origUPB             66582 non-null  int64  
 6   origLoanTerm        66582 non-null  int64  
 7   origDate            66582 non-null  object 
 8   origLTV             66582 non-null  float64
 9   origCLTV            66582 non-null  float64
 10  numBorrowers        66582 non-null  float64
 11  origDebtIncRatio    66582 non-null  float64
 12  loanPurp            66582 non-null  object 
 13  propState           66582 non-null  object 
 14  zipCode             66582 non-null  int64  
 15  pMIperct            66582 non-null  float64
 16  mort

## Remove columns not involved in 'Foreclosure or Not' query

In [5]:
df.drop(['Unnamed: 0','id','fredRate','monMatur','deliqGood','deliqBad','deliqMax'],1,inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66582 entries, 0 to 66581
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origChannel         66582 non-null  object 
 1   sellerName          66582 non-null  object 
 2   origIntRate         66582 non-null  float64
 3   origUPB             66582 non-null  int64  
 4   origLoanTerm        66582 non-null  int64  
 5   origDate            66582 non-null  object 
 6   origLTV             66582 non-null  float64
 7   origCLTV            66582 non-null  float64
 8   numBorrowers        66582 non-null  float64
 9   origDebtIncRatio    66582 non-null  float64
 10  loanPurp            66582 non-null  object 
 11  propState           66582 non-null  object 
 12  zipCode             66582 non-null  int64  
 13  pMIperct            66582 non-null  float64
 14  mortInsType         66582 non-null  float64
 15  bestCreditScore     66582 non-null  float64
 16  wors

### Check if any columns have a null value

In [7]:
df.apply(lambda x: x.isnull().sum(), axis=0)

origChannel           0
sellerName            0
origIntRate           0
origUPB               0
origLoanTerm          0
origDate              0
origLTV               0
origCLTV              0
numBorrowers          0
origDebtIncRatio      0
loanPurp              0
propState             0
zipCode               0
pMIperct              0
mortInsType           0
bestCreditScore       0
worstCreditScore      0
avgCreditScore        0
loanAge               0
mSA                   0
zeroBalCode           0
fmacRateMax           0
fmacRateMin           0
fmacRateAvg           0
fmacRateVolatility    0
rateDiffAbove         0
rateDiffBelow         0
rateDiffAvg           0
rateDiffAbovePct      0
rateDiffBelowPct      0
rateDiffAvgPct        0
dtype: int64

## Engineer originDate into Date features

In [8]:
df['origDate'] = pd.to_datetime(df['origDate'])

In [9]:
df['origYear'] = df['origDate'].dt.year
df['origMonth'] = df['origDate'].dt.month

In [10]:
df.drop(['origDate'],1,inplace=True)

In [11]:
df[['origYear','origMonth']].head()

,origYear,origMonth
0,2008,1
1,2009,1
2,2003,2
3,2004,7
4,2013,10


# Turn zero Balance Code Into binary "0" GOOD and "1" BAD

In [12]:
df.zeroBalCode.unique()

array([16.,  6.,  1.,  2.,  9.,  3., 15.])

In [13]:
df['zeroBalCode'] = df.zeroBalCode.replace([1,6,16],0)
df['zeroBalCode'] = df.zeroBalCode.replace([9,3,2,15],1)

In [14]:
df.zeroBalCode.unique()

array([0., 1.])

In [15]:
#Checking datatypes of individual feature
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,B,"BANK OF AMERICA, N.A.",6.500,248000,360,90.0,90.0,1.0,52.0,C,CA,953,25.0,2.0,681.0,681.0,681.0,127,33700,0.0,6.07,5.48,5.7580,0.107664,0.430,-1.020,0.7420,0.070840,-0.186131,0.128864,2008,1
1,R,"BANK OF AMERICA, N.A.",5.750,42000,360,85.0,85.0,1.0,48.0,P,NY,137,12.0,1.0,755.0,755.0,755.0,54,13780,0.0,5.12,4.96,5.0475,0.032258,0.630,-0.790,0.7025,0.123047,-0.159274,0.139178,2009,1
2,C,"WITMER FUNDING, LLC",5.875,216000,360,79.0,79.0,2.0,26.0,R,WI,531,0.0,0.0,753.0,748.0,750.5,138,33340,0.0,5.88,5.79,5.8425,0.015544,-0.005,-0.085,0.0325,-0.000850,-0.014680,0.005563,2003,2
3,C,"BANK OF AMERICA, N.A.",5.750,252000,360,79.0,79.0,1.0,50.0,R,CA,958,0.0,0.0,681.0,681.0,681.0,72,40900,1.0,6.21,5.98,6.0560,0.038462,-0.460,0.230,-0.3060,-0.074074,0.038462,-0.050528,2004,7
4,C,OTHER,3.625,102000,180,80.0,80.0,1.0,29.0,C,AL,351,0.0,0.0,665.0,665.0,665.0,49,13820,0.0,4.28,4.10,4.1920,0.043902,-0.655,0.475,-0.5670,-0.153037,0.115854,-0.135258,2013,10


# Review values for dates and slice (if needed) the timeframe you want

In [16]:
df.origYear.unique()

array([2008, 2009, 2003, 2004, 2013, 2002, 2006, 2007, 2005, 2001, 2017,
       2000, 2012, 2010, 2014, 2016, 2015, 1999, 2018, 2011, 2019],
      dtype=int64)

In [17]:
df['origMonth'].unique()

array([ 1,  2,  7, 10,  5,  9,  8,  6,  3, 12, 11,  4], dtype=int64)

In [18]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,mSA,zeroBalCode,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth
0,B,"BANK OF AMERICA, N.A.",6.500,248000,360,90.0,90.0,1.0,52.0,C,CA,953,25.0,2.0,681.0,681.0,681.0,127,33700,0.0,6.07,5.48,5.7580,0.107664,0.430,-1.020,0.7420,0.070840,-0.186131,0.128864,2008,1
1,R,"BANK OF AMERICA, N.A.",5.750,42000,360,85.0,85.0,1.0,48.0,P,NY,137,12.0,1.0,755.0,755.0,755.0,54,13780,0.0,5.12,4.96,5.0475,0.032258,0.630,-0.790,0.7025,0.123047,-0.159274,0.139178,2009,1
2,C,"WITMER FUNDING, LLC",5.875,216000,360,79.0,79.0,2.0,26.0,R,WI,531,0.0,0.0,753.0,748.0,750.5,138,33340,0.0,5.88,5.79,5.8425,0.015544,-0.005,-0.085,0.0325,-0.000850,-0.014680,0.005563,2003,2
3,C,"BANK OF AMERICA, N.A.",5.750,252000,360,79.0,79.0,1.0,50.0,R,CA,958,0.0,0.0,681.0,681.0,681.0,72,40900,1.0,6.21,5.98,6.0560,0.038462,-0.460,0.230,-0.3060,-0.074074,0.038462,-0.050528,2004,7
4,C,OTHER,3.625,102000,180,80.0,80.0,1.0,29.0,C,AL,351,0.0,0.0,665.0,665.0,665.0,49,13820,0.0,4.28,4.10,4.1920,0.043902,-0.655,0.475,-0.5670,-0.153037,0.115854,-0.135258,2013,10


# Coerce the column dtype (int in this case)

In [19]:
df = df.astype({'origLTV':'int','origCLTV':'int','numBorrowers':'int','origDebtIncRatio':'int',\
               'mortInsType':'int','bestCreditScore':'int','loanAge':'int',\
               'worstCreditScore':'int','avgCreditScore':'int','zeroBalCode':'int'})
#,'zeroBalCode':'int'

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66582 entries, 0 to 66581
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origChannel         66582 non-null  object 
 1   sellerName          66582 non-null  object 
 2   origIntRate         66582 non-null  float64
 3   origUPB             66582 non-null  int64  
 4   origLoanTerm        66582 non-null  int64  
 5   origLTV             66582 non-null  int32  
 6   origCLTV            66582 non-null  int32  
 7   numBorrowers        66582 non-null  int32  
 8   origDebtIncRatio    66582 non-null  int32  
 9   loanPurp            66582 non-null  object 
 10  propState           66582 non-null  object 
 11  zipCode             66582 non-null  int64  
 12  pMIperct            66582 non-null  float64
 13  mortInsType         66582 non-null  int32  
 14  bestCreditScore     66582 non-null  int32  
 15  worstCreditScore    66582 non-null  int32  
 16  avgC

<a id='MLReady'></a>

# Oversampling (Since we are starting with only 14.51% of our predicted class)

In [21]:
good = df.zeroBalCode.value_counts()[0]
bad = df.zeroBalCode.value_counts()[1]
perct_bad = round(bad/(good + bad)*100,2)
print(f'We have {perct_bad}% Foreclosures in our dataset')

We have 14.47% Foreclosures in our dataset


In [22]:
from sklearn.model_selection import train_test_split

training_features, test_features, \
training_target, test_target, = train_test_split(
    df.drop(['zeroBalCode'], axis=1)
    , df['zeroBalCode']
    , test_size = .1
    , random_state=12
)

In [23]:
# Further split the training data into training/test
x_train, x_val, y_train, y_val = train_test_split(
    training_features
    , training_target
    , test_size = .1
    ,random_state=12
)

In [24]:
# For the training data, randomly sample 
# ros = RandomOverSampler(sampling_strategy='minority')
# x_train_res, y_train_res = ros.fit_sample(x_train, y_train)

In [25]:
print('#############################################')
print('Before oversampling: "Closed" crushes "Default" and causes issues:')
print(training_target.value_counts())
# print('')
# print('Before oversampling: "Closed" and "Default" are equal')
# print(y_train_res.value_counts())

#############################################
Before oversampling: "Closed" crushes "Default" and causes issues:
0    51232
1     8691
Name: zeroBalCode, dtype: int64


# Recombine X_train_res, y_train_res so PyCaret can deal with one df

In [26]:
y_train.shape

(53930,)

In [27]:
df = x_train.copy()

In [28]:
df['zeroBalCode'] = y_train

In [29]:
df.shape

(53930, 32)

# Review metrics last time before run (to have record of starting values)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53930 entries, 38114 to 61123
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   origChannel         53930 non-null  object 
 1   sellerName          53930 non-null  object 
 2   origIntRate         53930 non-null  float64
 3   origUPB             53930 non-null  int64  
 4   origLoanTerm        53930 non-null  int64  
 5   origLTV             53930 non-null  int32  
 6   origCLTV            53930 non-null  int32  
 7   numBorrowers        53930 non-null  int32  
 8   origDebtIncRatio    53930 non-null  int32  
 9   loanPurp            53930 non-null  object 
 10  propState           53930 non-null  object 
 11  zipCode             53930 non-null  int64  
 12  pMIperct            53930 non-null  float64
 13  mortInsType         53930 non-null  int32  
 14  bestCreditScore     53930 non-null  int32  
 15  worstCreditScore    53930 non-null  int32  
 16  

In [31]:
df.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,mSA,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth,zeroBalCode
38114,R,"JPMORGAN CHASE BANK, NA",6.500,95000,180,78,78,1,29,C,WA,982,0.0,0,802,802,802,29,13380,7.20,7.10,7.1450,0.014085,-0.700,0.600,-0.6450,-0.097222,0.084507,-0.090273,2001,5,0
17823,C,"CITIMORTGAGE, INC.",5.750,78000,360,30,30,2,16,R,MT,591,0.0,0,771,771,771,70,13740,5.83,5.70,5.7540,0.022807,-0.080,-0.050,-0.0040,-0.013722,-0.008772,-0.000695,2004,9,0
7939,B,"BANK OF AMERICA, N.A.",6.375,112000,360,97,97,2,54,P,GA,305,35.0,1,698,665,681,110,12060,6.02,5.81,5.8760,0.036145,0.355,-0.565,0.4990,0.058970,-0.097246,0.084922,2003,12,0
24762,R,BISHOPS GATE RESIDENTIAL MORTGAGE TRUST,7.500,192000,360,78,78,2,45,C,CA,920,0.0,0,670,632,651,18,41740,7.02,6.81,6.8925,0.030837,0.480,-0.690,0.6075,0.068376,-0.101322,0.088139,2002,2,0
39023,B,REGIONS BANK,5.750,155000,360,54,54,2,43,C,CA,930,0.0,0,648,629,638,17,37100,6.03,5.83,5.9325,0.034305,-0.280,0.080,-0.1825,-0.046434,0.013722,-0.030763,2003,11,0


# Pycaret - Setup with categorical definition

### Use complex setup 

In [32]:
model_setup = setup(
    df
    , target = 'zeroBalCode' # PyCaret will list this as "Label"
    , normalize = True
    , transformation = False
    , pca = False 
    , ignore_low_variance = True # Variance is calculated using the ratio of unique values to the number of samples, and the ratio of the most common value to the frequency of the second most common value. 
    , ignore_features = None
    , handle_unknown_categorical = True
    , remove_outliers = True
    , remove_multicollinearity = True
    , polynomial_features = True
    , trigonometry_features = False
    , feature_selection = True
    , feature_interaction = True
    , fix_imbalance = True
    , silent = True
    , profile = False
    # outliers from the training data are removed using PCA linear dimensionality reduction using the Singular Value Decomposition technique.
#     , bin_numeric_features = [
#         'origIntRate'
#         , 'origUPB'
#         , 'origLTV'
#         , 'origDebtIncRatio'
#         , 'bestCreditScore'
#         , 'avgCreditScore'
#         , 'worstCreditScore'
#         ] # Set to True to bin numerics using K Means
    , categorical_features = [
         'origChannel'
        , 'loanPurp'
        , 'origYear'
        , 'origMonth'
        , 'mortInsType'
        , 'origLoanTerm'
        ]
    , high_cardinality_features = [
         'propState'
        , 'sellerName'
        , 'mSA'
        , 'zipCode'
    ]
    , high_cardinality_method = 'clustering'
    , numeric_features = [
        'origIntRate'
        , 'numBorrowers'
        , 'origUPB'
        , 'origLTV'
        , 'origCLTV'
        , 'pMIperct'
        , 'loanAge'
        , 'origDebtIncRatio'
        , 'worstCreditScore'
        , 'bestCreditScore'
        , 'avgCreditScore'
        , 'rateDiffAbove'
        , 'rateDiffBelow'
        , 'rateDiffAvg'
        , 'rateDiffAbovePct'
        , 'rateDiffBelowPct'
        , 'rateDiffAvgPct'
        , 'fmacRateMax'
        , 'fmacRateMin'
        , 'fmacRateAvg'
        , 'fmacRateVolatility'
    ]
)


Setup Succesfully Completed!


,Description,Value
0,session_id,1804
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(53930, 32)"
4,Missing Values,False
5,Numeric Features,23
6,Categorical Features,8
7,Ordinal Features,False
8,High Cardinality Features,True
9,High Cardinality Method,clustering


## Compare Models with no blacklist exclusions

In [33]:
model_results=compare_models(sort='Recall') #blacklist = ['tr','ransac'],'lar','par','huber','llar','lasso','en','ridge','omp','br','svm'])
model_results

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,SVM - Linear Kernel,0.6694,0.0000,0.7796,0.2779,0.4096,0.2461,0.3094,0.9297
1,Quadratic Discriminant Analysis,0.6413,0.7407,0.7586,0.2579,0.3843,0.2110,0.2723,0.4537
2,Naive Bayes,0.6277,0.7386,0.7487,0.2473,0.3718,0.1934,0.2534,0.1120
3,Ridge Classifier,0.7078,0.0000,0.7302,0.2985,0.4237,0.2716,0.3204,0.1061
4,Linear Discriminant Analysis,0.7076,0.7869,0.7296,0.2983,0.4234,0.2712,0.3199,0.9333
5,Logistic Regression,0.7096,0.7871,0.7266,0.2994,0.4241,0.2724,0.3203,1.1841
6,K Neighbors Classifier,0.6590,0.7107,0.6735,0.2527,0.3675,0.1954,0.2395,3.4454
7,Ada Boost Classifier,0.7991,0.7602,0.4161,0.3475,0.3786,0.2600,0.2616,12.0438
8,Decision Tree Classifier,0.7708,0.5919,0.3385,0.2741,0.3028,0.1675,0.1689,3.8231
9,Gradient Boosting Classifier,0.8432,0.7931,0.3064,0.4521,0.3650,0.2793,0.2864,57.5210


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=-1, penalty='l2',
              power_t=0.5, random_state=1804, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

## Create models 

### LGBM Classifier

In [44]:
lgbm = create_model('lightgbm', fold=10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8556,0.8063,0.2462,0.5200,0.3342,0.2646,0.2880
1,0.8520,0.8101,0.2330,0.4940,0.3166,0.2454,0.2673
2,0.8640,0.8238,0.2500,0.5893,0.3511,0.2887,0.3220
3,0.8608,0.8141,0.2619,0.5565,0.3561,0.2893,0.3152
4,0.8625,0.8186,0.2751,0.5664,0.3704,0.3034,0.3285
5,0.8611,0.8192,0.2808,0.5543,0.3728,0.3040,0.3263
6,0.8525,0.8229,0.2386,0.4980,0.3227,0.2512,0.2727
7,0.8581,0.8120,0.2386,0.5408,0.3311,0.2649,0.2928
8,0.8575,0.8146,0.2500,0.5344,0.3406,0.2724,0.2972
9,0.8561,0.8030,0.2273,0.5263,0.3175,0.2509,0.2788


## Evaluate Models

In [47]:
evaluate_model(lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

## Tune Model

In [49]:
tuned_lgbm = tune_model(lgbm, optimize = 'Recall')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8366,0.7607,0.2973,0.4220,0.3489,0.2587,0.2639
1,0.8246,0.7551,0.2803,0.3728,0.3200,0.2217,0.2246
2,0.8336,0.7653,0.3049,0.4118,0.3504,0.2574,0.2612
3,0.8360,0.7677,0.3207,0.4236,0.3650,0.2729,0.2764
4,0.8335,0.7731,0.2979,0.4089,0.3447,0.2520,0.2562
5,0.8279,0.7539,0.2960,0.3881,0.3358,0.2391,0.2420
6,0.8305,0.7611,0.2860,0.3953,0.3319,0.2376,0.2417
7,0.8332,0.7593,0.3030,0.4103,0.3486,0.2554,0.2593
8,0.8293,0.7518,0.2955,0.3939,0.3377,0.2420,0.2453
9,0.8299,0.7423,0.2481,0.3808,0.3005,0.2085,0.2147


In [51]:
evaluate_model(tuned_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [50]:
bagged_lgbm = ensemble_model(lgbm)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8598,0.8096,0.2595,0.5502,0.3526,0.2852,0.3107
1,0.8492,0.8131,0.2235,0.4739,0.3037,0.2312,0.2518
2,0.8612,0.8265,0.2443,0.5658,0.3413,0.2771,0.3078
3,0.8572,0.8135,0.2524,0.5299,0.3419,0.2730,0.2967
4,0.8578,0.8223,0.2524,0.5341,0.3428,0.2743,0.2987
5,0.8636,0.8192,0.2941,0.5699,0.3880,0.3199,0.3422
6,0.8528,0.8245,0.2481,0.5000,0.3316,0.2593,0.2795
7,0.8564,0.8124,0.2443,0.5265,0.3338,0.2652,0.2899
8,0.8569,0.8135,0.2367,0.5319,0.3277,0.2606,0.2875
9,0.8564,0.8051,0.2273,0.5286,0.3179,0.2516,0.2798


In [52]:
evaluate_model(bagged_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [54]:
boosted_lgbm = ensemble_model(lgbm, method='Boosting')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8528,0.7917,0.2500,0.5000,0.3333,0.2608,0.2807
1,0.8489,0.7975,0.2367,0.4735,0.3157,0.2412,0.2596
2,0.8573,0.8094,0.2822,0.5284,0.3679,0.2957,0.3143
3,0.8522,0.8025,0.2751,0.4949,0.3537,0.2778,0.2931
4,0.8550,0.8093,0.2732,0.5125,0.3564,0.2832,0.3010
5,0.8578,0.8055,0.2865,0.5298,0.3719,0.2997,0.3177
6,0.8472,0.7950,0.2348,0.4627,0.3116,0.2358,0.2530
7,0.8525,0.7969,0.2405,0.4980,0.3244,0.2527,0.2739
8,0.8547,0.8044,0.2538,0.5134,0.3397,0.2684,0.2895
9,0.8461,0.7835,0.2178,0.4528,0.2941,0.2195,0.2381


In [55]:
evaluate_model(boosted_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [56]:
final_lgbm = finalize_model(lgbm)

In [59]:
evaluate_model(final_lgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# Predict models

In [60]:
predicted_final_lgbm = predict_model(final_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8803,0.8838,0.2494,0.7977,0.3801,0.3333,0.4033


In [61]:
predicted_lgbm = predict_model(lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8652,0.8121,0.1955,0.636,0.2991,0.247,0.3004


In [62]:
predicted_lgbm_tuned = predict_model(tuned_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8334,0.7494,0.2848,0.4058,0.3347,0.2428,0.2478


In [63]:
predicted_lgbm_bagged = predict_model(bagged_lgbm)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Bagging Classifier,0.8657,0.8147,0.1791,0.6607,0.2818,0.2338,0.2955


# The golden moment... put 10% sample test data through the models...

# Recombine test_features, test_target at first train/test/split
 This "test" set was split BEFORE Oversampling. PyCaret takes the "train" from this split and splits it internally again. Since we have no control of how PyCaret does this we needed to do a train/test split before we input the training set into PyCaret. Now we will recombine the test_features and test_target into one new df

In [64]:
test_features.shape

(6659, 31)

In [65]:
test_target.shape

(6659,)

In [66]:
test_df = test_features.copy()

In [67]:
test_df['zeroBalCode'] = test_target

In [68]:
test_df.shape

(6659, 32)

In [69]:
totalTestCount = test_df.zeroBalCode.value_counts().sum()
posTestCount = test_df.zeroBalCode.value_counts()[1]
print('#############################################')
print('This "Test" set has not been oversamplied')
print(test_df.zeroBalCode.value_counts())
print(f'Positive samples are {round(posTestCount/totalTestCount*100,2)}%')

#############################################
This "Test" set has not been oversamplied
0    5715
1     944
Name: zeroBalCode, dtype: int64
Positive samples are 14.18%


## Create Confusion Matrix output functions

In [70]:
def calc_confusion(row):
    if ((row['zeroBalCode'] == 0) & (row['Label'] == 0)):
        value = 'TrueNegative'
    elif ((row['zeroBalCode'] == 0) & (row['Label'] == 1)):
        value = 'FalsePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 1)):
        value = 'TruePositive'
    elif ((row['zeroBalCode'] == 1) & (row['Label'] == 0)):
        value = 'FalseNegative'
    else:
        value = 'Undefined'
    return value

In [71]:
def cM(df):
    print(f'           ##############################')
    print(f'           #             #              #')
    print(f'         0 #    {df["TrueNegative"]}     #     {df["FalsePositive"]}      #')
    print(f'  True     #             #              #')
    print(f'  Class    ##############################')
    print(f'           #             #              #')
    print(f'         1 #    {df["FalseNegative"]}      #     {df["TruePositive"]}      #')
    print(f'           #             #              #')
    print(f'           ##############################')
    print(f'                  0              1        ')
    print(f'                  Predicted Class           ')


In [72]:
def cMCalc():
    # 0 or 2 = Predict equals truth, 1 = Predict doesn't equal truth
    totalNegative = confusionMatrix['TrueNegative'] + confusionMatrix['FalseNegative']
    totalPositive = confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']
    total = totalNegative + totalPositive
    trueNegativePct = confusionMatrix['TrueNegative'] / total * 100
    falseNegativePct = confusionMatrix['FalseNegative'] / total * 100
    truePositivePct = confusionMatrix['TruePositive'] / total * 100
    falsePositivePct = confusionMatrix['FalsePositive'] / total * 100

    # Accuracy Calculation
    accuracy = (confusionMatrix['TruePositive'] + confusionMatrix['TrueNegative'])/(totalPositive + totalNegative)
    # Recall Calculation
    recall = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalseNegative']))
    # Precision Calculation
    precision = (confusionMatrix['TruePositive']/(confusionMatrix['TruePositive'] + confusionMatrix['FalsePositive']))

    print(f'TrueNegative:  {round(trueNegativePct,2)}%')
    print(f'FalseNegative:  {round(falseNegativePct,2)}%')
    print(f'TruePositive:   {round(truePositivePct,2)}%')
    print(f'FalsePositive: {round(falsePositivePct,2)}%')
    print(f'#####################################################')
    print(f'Accuracy: {round(accuracy,2)*100}%   Recall: {round(recall,2)*100}%   Precision: {round(precision,2)*100}%')
    print(f'#####################################################')

# Evaluate Models

In [74]:
predictions = predict_model(final_lgbm, data=test_df)

In [75]:
pd.set_option('max_columns',None)
predictions.head()

,origChannel,sellerName,origIntRate,origUPB,origLoanTerm,origLTV,origCLTV,numBorrowers,origDebtIncRatio,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore,loanAge,mSA,fmacRateMax,fmacRateMin,fmacRateAvg,fmacRateVolatility,rateDiffAbove,rateDiffBelow,rateDiffAvg,rateDiffAbovePct,rateDiffBelowPct,rateDiffAvgPct,origYear,origMonth,zeroBalCode,Label,Score
0,C,NETBANK FUNDING SERVICES,6.625,175000,360,50,50,2,42,R,MA,23,0.0,0,745,720,732,5,14460,6.57,6.34,6.485,0.036278,0.055,-0.285,0.140,0.008371,-0.044953,0.021588,2002,7,0,0,0.0022
1,R,OTHER,4.875,158000,180,45,45,2,8,R,OH,437,0.0,0,805,754,779,138,0,5.70,5.31,5.484,0.073446,-0.825,0.435,-0.609,-0.144737,0.081921,-0.111050,2003,5,0,0,0.0122
2,C,"JPMORGAN CHASE BANK, NA",5.250,102000,360,91,91,1,26,R,OK,741,25.0,1,753,753,753,94,46140,6.34,6.14,6.264,0.032573,-1.090,0.890,-1.014,-0.171924,0.144951,-0.161877,2003,8,0,0,0.0909
3,B,"PNC BANK, N.A.",6.000,187000,360,49,49,2,62,C,CA,919,0.0,0,775,775,775,44,41740,5.97,5.85,5.916,0.020513,0.030,-0.150,0.084,0.005025,-0.025641,0.014199,2003,1,0,0,0.0255
4,C,"CITIMORTGAGE, INC.",6.625,135000,360,40,40,2,31,P,CA,935,0.0,0,788,782,785,32,31100,6.78,6.62,6.682,0.024169,-0.155,-0.005,-0.057,-0.022861,-0.000755,-0.008530,2006,6,0,0,0.0085


In [76]:
results = predictions[['zeroBalCode','Label','Score']]

In [77]:
predictions.Label.value_counts()

0    6656
1       3
Name: Label, dtype: int64

In [78]:
results['Confusion'] = results.apply(calc_confusion, axis=1)

In [79]:
confusionMatrix = results.Confusion.value_counts().to_dict()

In [80]:
cM(confusionMatrix)

           ##############################
           #             #              #
         0 #    5713     #     2      #
  True     #             #              #
  Class    ##############################
           #             #              #
         1 #    943      #     1      #
           #             #              #
           ##############################
                  0              1        
                  Predicted Class           


In [81]:
cMCalc()

TrueNegative:  85.79%
FalseNegative:  14.16%
TruePositive:   0.02%
FalsePositive: 0.03%
#####################################################
Accuracy: 86.0%   Recall: 0.0%   Precision: 33.0%
#####################################################


In [82]:
#save_model(final_nb, 'np.final5pct_092020')